From: Shilpi

In [1]:
import warnings
warnings.filterwarnings('ignore')

from sigpyproc.readers import FilReader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import dates
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator
from datetime import datetime, timedelta
import astropy.units as u
from astropy.time import Time
from astropy.visualization import ImageNormalize, PercentileInterval

plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'

# print(plt.get_backend())
path = '/home/mnedal/data/ilofar'

In [ ]:
stokes = 'V'
filename = f'{path}/Sun357_20240514_stokes{stokes}.fil'

In [8]:
a = FilReader(filename) # header
header = a.header.to_dict()

In [9]:
header

{'filename': '/home/mnedal/data/Sun357_2025-05-07_checkRFI.fil',
 'data_type': 'filterbank',
 'nchans': 488,
 'foff': -0.1953125,
 'fch1': 200.0,
 'nbits': 32,
 'tsamp': 0.00131072,
 'tstart': 60802.52152777409,
 'nsamples': 640869,
 'nifs': 1,
 'coord': <SkyCoord (ICRS): (ra, dec) in deg
     (0., 0.)>,
 'azimuth': <Angle 277.86152637 deg>,
 'zenith': <Angle 76.94861002 deg>,
 'telescope': 'MeerKAT',
 'backend': 'FAKE',
 'source': 'Sun357',
 'frame': 'topocentric',
 'ibeam': 0,
 'nbeams': 0,
 'dm': 0,
 'period': 0,
 'accel': 0,
 'signed': False,
 'rawdatafile': 'unknown',
 'stream_info': {'entries': [{'filename': '/home/mnedal/data/Sun357_2025-05-07_checkRFI.fil',
    'hdrlen': 347,
    'datalen': 1250976288,
    'nsamples': 640869,
    'tstart': 60802.52152777409,
    'tsamp': 0.00131072}]},
 'basename': 'Sun357_2025-05-07_checkRFI',
 'extension': '.fil',
 'telescope_id': 64,
 'machine_id': 0,
 'bandwidth': 95.3125,
 'ftop': 200.09765625,
 'fbottom': 104.78515625,
 'fcenter': 152.441

In [5]:
tstart_obs_str = Time(a.header.tstart, format='mjd').iso
tstart = datetime.strptime(tstart_obs_str, '%Y-%m-%d %H:%M:%S.%f')

tstart_str = '2024-05-14 16:35:00'
tend_str   = '2024-05-14 17:50:00'
datetime1 = datetime.strptime(tstart_str, '%Y-%m-%d %H:%M:%S')
datetime2 = datetime.strptime(tend_str, '%Y-%m-%d %H:%M:%S')

print(tstart, datetime1, datetime2, sep='\n')

time_difference1 = datetime1 - tstart
time_difference2 = datetime2 - datetime1

secondsFromStart = time_difference1.total_seconds()
totalTime = time_difference2.total_seconds()

print(secondsFromStart, totalTime, sep='\n')

2025-05-07 12:31:00
2024-05-14 16:35:00
2024-05-14 17:50:00
-30916560.0
4500.0


In [ ]:
# downsampling the data to time resolution of 0.5 s. the time resolution to 0.5 s > 500 ms. And 500/1.31 ms = ~381. So tfactor should be 381.
data = a.read_block(int(secondsFromStart / a.header.tsamp), int(totalTime / a.header.tsamp))
# np.save(f'{path}/realta_ilofar.npy', data)

In [ ]:
# data.nbytes/(1024*1024*1024) # data size in GB

In [ ]:
print(data.shape, data.shape[1]*a.header.tsamp, sep='\n')

In [ ]:
# define time cadense in ms.
Tres = 10                  # time resolution of n millisecond
tres = a.header.tsamp*1e3  # time resolution in header

# downsampling data applied by summing sequential sample/channels and averaging the data
data2 = data.downsample(tfactor=int(Tres/tres))
data2.shape

In [ ]:
np.save(f'{path}/res_{Tres}ms_ilofar_stokes{stokes}.npy', data2)

In [ ]:
# Creating time axis
tstart = Time(data2.header.tstart, format='mjd')                 # tstart.iso will tell the time in format yyyy-mm-dd hh:mm:ss
# tarray = tstart + (np.arange(data2.shape[1])*(Tres/1000)*u.s)  # Creating the time array for Tres time res.
tarray = tstart + (np.arange(data2.shape[1])*data2.header.tsamp*u.s) # Creating the time array for realta time resolution
tarray[-1].iso

In [ ]:
dt = datetime.strptime(tarray[1].iso, '%Y-%m-%d %H:%M:%S.%f') - datetime.strptime(tarray[0].iso, '%Y-%m-%d %H:%M:%S.%f')
dt.total_seconds()*1000 # in ms.

In [ ]:
Tarray = [datetime.strptime(t.iso, '%Y-%m-%d %H:%M:%S.%f') for t in tarray] # Converting the array to datetime object
Tarray = np.array(Tarray)
np.save(f'{path}/time_{Tres}ms_stokes{stokes}.npy', Tarray)

In [ ]:
print(Tarray[0], Tarray[1], Tarray[-1], sep='\n')

In [ ]:
print(Tarray[1]-Tarray[0], Tarray[-1]-Tarray[-2], sep='\n')

In [ ]:
# export the frequency axis
freqs = data2.header.chan_freqs
np.save(f'{path}/freq_{Tres}ms_stokes{stokes}.npy', freqs)

---

### Import the exported data files

In [ ]:
# import the data files
Tres = 10 # in ms.
stokes = 'V'
data = np.load(f'{path}/res_{Tres}ms_ilofar_stokes{stokes}.npy')
time = np.load(f'{path}/time_{Tres}ms_stokes{stokes}.npy', allow_pickle=True)
freqs = np.load(f'{path}/freq_{Tres}ms_stokes{stokes}.npy')

In [ ]:
(time[1] - time[0]).total_seconds()

In [ ]:
def freq_axis(freqs):
    '''
    Introduce gaps in the frequency axis of I-LOFAR REALTA data.
    '''
    gap1 = np.flipud(freqs[288]+(np.arange(59)*0.390625))
    gap2 = np.flipud(freqs[88]+(np.arange(57)*0.390625))
    ax_shape = 59+57-1
    new_freq = np.zeros(ax_shape+freqs.shape[0])
    
    new_freq[0:88] = freqs[0:88]
    new_freq[88:145]  = gap2[:57]
    new_freq[145:345] = freqs[88:288]
    new_freq[345:404] = gap1[:59]
    new_freq[404:] = freqs[289:]
    
    return new_freq

In [ ]:
new_freq = freq_axis(freqs)

data = np.log10(data)
data[np.where(np.isinf(data)==True)] = 0.0

data2 = np.empty((new_freq.shape[0], data.shape[1]))    
data2[:] = np.NaN
data2[0:88] = data[0:88]
data2[145:345] = data[88:288]
data2[404:] = data[289:]

# times_mpl = [mdates.date2num(t) for t in time]

In [ ]:
freq_mode3 = np.linspace(10, 90, 199)
freq_mode5 = np.linspace(110, 190, 200)
freq_mode7 = np.linspace(210, 270, 88)

# downsample the data
df_mode3 = pd.DataFrame(data=data2[404:].T, columns=freq_mode3[::-1])
df_mode5 = pd.DataFrame(data=data2[145:345].T, columns=freq_mode5[::-1])
df_mode7 = pd.DataFrame(data=data2[:88].T, columns=freq_mode7[::-1])

In [ ]:
print(df_mode3.shape, df_mode5.shape, df_mode7.shape, sep='\n')

In [ ]:
# Save the dataframes as a pickle files
df_mode3.to_pickle(f'{path}/df_mode3_{Tres}ms_stokes{stokes}.pkl')
df_mode5.to_pickle(f'{path}/df_mode5_{Tres}ms_stokes{stokes}.pkl')
df_mode7.to_pickle(f'{path}/df_mode7_{Tres}ms_stokes{stokes}.pkl')

---

In [ ]:
# Load the dataframes from the pickle files
Tres = 10
stokes = 'V'

df_mode3 = pd.read_pickle(f'{path}/df_mode3_{Tres}ms_stokes{stokes}.pkl')
df_mode5 = pd.read_pickle(f'{path}/df_mode5_{Tres}ms_stokes{stokes}.pkl')
df_mode7 = pd.read_pickle(f'{path}/df_mode7_{Tres}ms_stokes{stokes}.pkl')
time = np.load(f'{path}/time_{Tres}ms_stokes{stokes}.npy', allow_pickle=True)
# freqs = np.load(f'{path}/freq_{Tres}ms_stokes{stokes}.npy')

In [ ]:
# Ensure the 'time' column is in datetime format
df_mode3.index = pd.to_datetime(time)
df_mode5.index = pd.to_datetime(time)
df_mode7.index = pd.to_datetime(time)

In [ ]:
print(df_mode3.shape, df_mode5.shape, df_mode7.shape, sep='\n')

In [ ]:
display(df_mode3.head(3)), display(df_mode3.tail(3))

---

In [ ]:
df_mode3 = df_mode3.resample('1S').mean()
df_mode5 = df_mode5.resample('1S').mean()
df_mode7 = df_mode7.resample('1S').mean()

In [ ]:
fig_title = str(time[0].date()).replace("-","") + f'_Tres_{Tres}ms_stokes{stokes}'

In [ ]:
del time
del freqs
del data
del data2
del new_freq

In [ ]:
if stokes == 'I':
    min_range_mode3 = np.nanpercentile(df_mode3.values, 10)
    max_range_mode3 = np.nanpercentile(df_mode3.values, 97)
    
    min_range_mode5 = np.nanpercentile(df_mode5.values, 10)
    max_range_mode5 = np.nanpercentile(df_mode5.values, 97)
    
    min_range_mode7 = np.nanpercentile(df_mode7.values, 10)
    max_range_mode7 = np.nanpercentile(df_mode7.values, 97)

elif stokes == 'V':
    min_range_mode3 = 0.1
    min_range_mode5 = 0.1
    min_range_mode7 = 0.1
    
    max_range_mode3 = 5
    max_range_mode5 = 5
    max_range_mode7 = 5

fig = plt.figure(figsize=[12,6])
ax = fig.add_subplot(111)
ax.pcolormesh(df_mode3.index, df_mode3.columns, df_mode3.values.T,
                    vmin=min_range_mode3, vmax=max_range_mode3,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode5.index, df_mode5.columns, df_mode5.values.T,
                    vmin=min_range_mode5, vmax=max_range_mode5,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode7.index, df_mode7.columns, df_mode7.values.T,
                    vmin=min_range_mode7, vmax=max_range_mode7,
                    cmap='RdYlBu_r')
ax.set_yscale('log')

# Define the custom ticks
custom_ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]
ax.set_yticks(custom_ticks)
ax.set_yticklabels([str(tick) for tick in custom_ticks])
ax.xaxis.set_minor_locator(AutoMinorLocator(n=10))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_ylim(ax.get_ylim()[::-1])
# fig.savefig(f'{path}/plots/realta_ilofar_{fig_title}.png', format='png', bbox_inches='tight')
# fig.savefig(f'{path}/plots/realta_ilofar_{fig_title}.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
# remove the const background
mode3_new = df_mode3.values - np.tile(np.nanmean(df_mode3.values,0), (df_mode3.values.shape[0],1))
mode5_new = df_mode5.values - np.tile(np.nanmean(df_mode5.values,0), (df_mode5.values.shape[0],1))x
mode7_new = df_mode7.values - np.tile(np.nanmean(df_mode7.values,0), (df_mode7.values.shape[0],1))

In [ ]:
min_range = -0.5
max_range = 2

fig = plt.figure(figsize=[12,6])
ax = fig.add_subplot(111)
ax.pcolormesh(df_mode3.index, df_mode3.columns, mode3_new.T,
              vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode5.index, df_mode5.columns, mode5_new.T,
              vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode7.index, df_mode7.columns, mode7_new.T,
              vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.set_yscale('log')

# Define the custom ticks
custom_ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]
ax.set_yticks(custom_ticks)
ax.set_yticklabels([str(tick) for tick in custom_ticks])
ax.xaxis.set_minor_locator(AutoMinorLocator(n=10))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_ylim(ax.get_ylim()[::-1])
fig.savefig(f'{path}/plots/realta_ilofar_{fig_title}_backgroundSubtracted.png', format='png', bbox_inches='tight')
plt.show()

### Zoom-in certain features

In [ ]:
Tres = 10
stokes = 'V'

# Load the dataframes from the pickle files
df_mode3 = pd.read_pickle(f'{path}/df_mode3_{Tres}ms_stokes{stokes}.pkl')
df_mode5 = pd.read_pickle(f'{path}/df_mode5_{Tres}ms_stokes{stokes}.pkl')
df_mode7 = pd.read_pickle(f'{path}/df_mode7_{Tres}ms_stokes{stokes}.pkl')
time = np.load(f'{path}/time_{Tres}ms_stokes{stokes}.npy', allow_pickle=True)

freqs_mode3 = df_mode3.columns
freqs_mode5 = df_mode5.columns
freqs_mode7 = df_mode7.columns

# remove the const background
mode3_new = df_mode3.values - np.tile(np.nanmean(df_mode3.values,0), (df_mode3.values.shape[0],1))
mode5_new = df_mode5.values - np.tile(np.nanmean(df_mode5.values,0), (df_mode5.values.shape[0],1))
mode7_new = df_mode7.values - np.tile(np.nanmean(df_mode7.values,0), (df_mode7.values.shape[0],1))

del df_mode3
del df_mode5
del df_mode7

df_mode3 = pd.DataFrame(data=mode3_new, columns=freqs_mode3)
df_mode5 = pd.DataFrame(data=mode5_new, columns=freqs_mode5)
df_mode7 = pd.DataFrame(data=mode7_new, columns=freqs_mode7)

# Ensure the 'time' column is in datetime format
df_mode3.index = pd.to_datetime(time)
df_mode5.index = pd.to_datetime(time)
df_mode7.index = pd.to_datetime(time)

In [ ]:
# downsample the data
df_mode3 = df_mode3.resample('100ms').mean()
df_mode5 = df_mode5.resample('100ms').mean()
df_mode7 = df_mode7.resample('100ms').mean()

In [ ]:
df_mode3.shape

In [ ]:
# Slice the DataFrame between start_date and end_date
start_date = '2024-05-14 16:45:00'
end_date   = '2024-05-14 17:00:00'

df_mode3_slice = df_mode3.loc[start_date:end_date]
df_mode5_slice = df_mode5.loc[start_date:end_date]
df_mode7_slice = df_mode7.loc[start_date:end_date]

In [ ]:
df_mode3_slice.shape

In [ ]:
fig = plt.figure(figsize=[12,6])
ax = fig.add_subplot(111)
ax.pcolormesh(df_mode3_slice.index, df_mode3_slice.columns, df_mode3_slice.T,
                    vmin=np.percentile(df_mode3_slice, 10), vmax=np.percentile(df_mode3_slice, 97),
                    # vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode5_slice.index, df_mode5_slice.columns, df_mode5_slice.T,
                    vmin=np.percentile(df_mode5_slice, 10), vmax=np.percentile(df_mode5_slice, 97),
              # vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode7_slice.index, df_mode7_slice.columns, df_mode7_slice.T,
                    vmin=np.percentile(df_mode7_slice, 10), vmax=np.percentile(df_mode7_slice, 97),
              # vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.set_yscale('log')

# Define the custom ticks
custom_ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]
ax.set_yticks(custom_ticks)
ax.set_yticklabels([str(tick) for tick in custom_ticks])
ax.xaxis.set_minor_locator(AutoMinorLocator(n=5))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_ylim(ax.get_ylim()[::-1])
# fig.savefig(f'{path}/plots/realta_ilofar_{fig_title}_backgroundSubtracted.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
# Slice the DataFrame between start_date and end_date
start_date = None #'2024-05-14 17:30:00'
# end_date   = '2024-05-14 17:13:00'

df_mode3_slice = df_mode3.loc[start_date:]
df_mode5_slice = df_mode5.loc[start_date:]
df_mode7_slice = df_mode7.loc[start_date:]

In [ ]:
df_mode3_slice.shape

In [ ]:
feature_type = 'all_dyspec' # typeII, typeIII, all_dyspec

fig = plt.figure(figsize=[12,6])
ax = fig.add_subplot(111)
img = ax.pcolormesh(df_mode3_slice.index, df_mode3_slice.columns, df_mode3_slice.T,
                    vmin=np.nanpercentile(df_mode3_slice, 10), vmax=np.nanpercentile(df_mode3_slice, 97),
                    # vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode5_slice.index, df_mode5_slice.columns, df_mode5_slice.T,
                    vmin=np.nanpercentile(df_mode5_slice, 10), vmax=np.nanpercentile(df_mode5_slice, 97),
              # vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode7_slice.index, df_mode7_slice.columns, df_mode7_slice.T,
                    vmin=np.nanpercentile(df_mode7_slice, 10), vmax=np.nanpercentile(df_mode7_slice, 97),
              # vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.set_yscale('log')
fig.colorbar(img, pad=0.02)

# Define the custom ticks
custom_ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]
ax.set_yticks(custom_ticks)
ax.set_yticklabels([str(tick) for tick in custom_ticks])
ax.xaxis.set_minor_locator(AutoMinorLocator(n=5))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_ylim(ax.get_ylim()[::-1])
fig.savefig(f'{path}/plots/ilofar_{str(df_mode3_slice.index[0].date())}_{feature_type}_stokes{stokes}_backgroundSubtracted.png', format='png', bbox_inches='tight')
plt.show()

---

### Calculate the degree of circular polarization

In [ ]:
# Load the dataframes from the pickle files
Tres = 10

df_mode3_stokesI = pd.read_pickle(f'{path}/df_mode3_{Tres}ms_stokesI.pkl')
df_mode5_stokesI = pd.read_pickle(f'{path}/df_mode5_{Tres}ms_stokesI.pkl')
df_mode7_stokesI = pd.read_pickle(f'{path}/df_mode7_{Tres}ms_stokesI.pkl')
time_stokesI = np.load(f'{path}/time_{Tres}ms_stokesI.npy', allow_pickle=True)

freqs_mode3_stokesI = df_mode3_stokesI.columns
freqs_mode5_stokesI = df_mode5_stokesI.columns
freqs_mode7_stokesI = df_mode7_stokesI.columns

# remove the const background
mode3_new_stokesI = df_mode3_stokesI.values - np.tile(np.nanmean(df_mode3_stokesI.values,0), (df_mode3_stokesI.values.shape[0],1))
mode5_new_stokesI = df_mode5_stokesI.values - np.tile(np.nanmean(df_mode5_stokesI.values,0), (df_mode5_stokesI.values.shape[0],1))
mode7_new_stokesI = df_mode7_stokesI.values - np.tile(np.nanmean(df_mode7_stokesI.values,0), (df_mode7_stokesI.values.shape[0],1))

del df_mode3_stokesI
del df_mode5_stokesI
del df_mode7_stokesI

df_mode3_stokesI = pd.DataFrame(data=mode3_new_stokesI, columns=freqs_mode3_stokesI)
df_mode5_stokesI = pd.DataFrame(data=mode5_new_stokesI, columns=freqs_mode5_stokesI)
df_mode7_stokesI = pd.DataFrame(data=mode7_new_stokesI, columns=freqs_mode7_stokesI)

# Ensure the 'time' column is in datetime format
df_mode3_stokesI.index = pd.to_datetime(time_stokesI)
df_mode5_stokesI.index = pd.to_datetime(time_stokesI)
df_mode7_stokesI.index = pd.to_datetime(time_stokesI)

# # downsample the data
# df_mode3_stokesI = df_mode3_stokesI.resample('100ms').mean()
# df_mode5_stokesI = df_mode5_stokesI.resample('100ms').mean()
# df_mode7_stokesI = df_mode7_stokesI.resample('100ms').mean()

In [ ]:
df_mode3_stokesV = pd.read_pickle(f'{path}/df_mode3_{Tres}ms_stokesV.pkl')
df_mode5_stokesV = pd.read_pickle(f'{path}/df_mode5_{Tres}ms_stokesV.pkl')
df_mode7_stokesV = pd.read_pickle(f'{path}/df_mode7_{Tres}ms_stokesV.pkl')
time_stokesV = np.load(f'{path}/time_{Tres}ms_stokesV.npy', allow_pickle=True)

freqs_mode3_stokesV = df_mode3_stokesV.columns
freqs_mode5_stokesV = df_mode5_stokesV.columns
freqs_mode7_stokesV = df_mode7_stokesV.columns

# remove the const background
mode3_new_stokesV = df_mode3_stokesV.values - np.tile(np.nanmean(df_mode3_stokesV.values,0), (df_mode3_stokesV.values.shape[0],1))
mode5_new_stokesV = df_mode5_stokesV.values - np.tile(np.nanmean(df_mode5_stokesV.values,0), (df_mode5_stokesV.values.shape[0],1))
mode7_new_stokesV = df_mode7_stokesV.values - np.tile(np.nanmean(df_mode7_stokesV.values,0), (df_mode7_stokesV.values.shape[0],1))

del df_mode3_stokesV
del df_mode5_stokesV
del df_mode7_stokesV

df_mode3_stokesV = pd.DataFrame(data=mode3_new_stokesV, columns=freqs_mode3_stokesV)
df_mode5_stokesV = pd.DataFrame(data=mode5_new_stokesV, columns=freqs_mode5_stokesV)
df_mode7_stokesV = pd.DataFrame(data=mode7_new_stokesV, columns=freqs_mode7_stokesV)

# Ensure the 'time' column is in datetime format
df_mode3_stokesV.index = pd.to_datetime(time_stokesV)
df_mode5_stokesV.index = pd.to_datetime(time_stokesV)
df_mode7_stokesV.index = pd.to_datetime(time_stokesV)

# # downsample the data
# df_mode3_stokesV = df_mode3_stokesV.resample('100ms').mean()
# df_mode5_stokesV = df_mode5_stokesV.resample('100ms').mean()
# df_mode7_stokesV = df_mode7_stokesV.resample('100ms').mean()

In [ ]:
df_mode3_stokesVI = df_mode3_stokesV / df_mode3_stokesI
df_mode5_stokesVI = df_mode5_stokesV / df_mode5_stokesI
df_mode7_stokesVI = df_mode7_stokesV / df_mode7_stokesI

# downsample the data
# df_mode3_stokesVI = df_mode3_stokesVI.resample('100ms').apply(np.nanmean)
# df_mode5_stokesVI = df_mode5_stokesVI.resample('100ms').apply(np.nanmean)
# df_mode7_stokesVI = df_mode7_stokesVI.resample('100ms').apply(np.nanmean)

df_mode3_stokesVI = df_mode3_stokesVI.resample('100ms').mean()
df_mode5_stokesVI = df_mode5_stokesVI.resample('100ms').mean()
df_mode7_stokesVI = df_mode7_stokesVI.resample('100ms').mean()

In [ ]:
# Slice the DataFrame between start_date and end_date
start_date = None #'2024-05-14 17:30:00'

df_mode3_stokesVI_slice = df_mode3_stokesVI.loc[start_date:]
df_mode5_stokesVI_slice = df_mode5_stokesVI.loc[start_date:]
df_mode7_stokesVI_slice = df_mode7_stokesVI.loc[start_date:]

In [ ]:
print('Max of Max:', max(np.nanmax(df_mode3_stokesVI_slice), np.nanmax(df_mode5_stokesVI_slice), np.nanmax(df_mode7_stokesVI_slice)))
print('Min of Min', min(np.nanmin(df_mode3_stokesVI_slice), np.nanmin(df_mode5_stokesVI_slice), np.nanmin(df_mode7_stokesVI_slice)))

In [ ]:
min_range = -1178161
max_range = 16520625

fig = plt.figure(figsize=[12,6])
ax = fig.add_subplot(111)
img = ax.pcolormesh(df_mode3_stokesVI_slice.index, df_mode3_stokesVI_slice.columns, df_mode3_stokesVI_slice.T,
                    vmin=np.nanpercentile(df_mode3_stokesVI_slice, 10), vmax=np.nanpercentile(df_mode3_stokesVI_slice, 97),
                    # vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode5_stokesVI_slice.index, df_mode5_stokesVI_slice.columns, df_mode5_stokesVI_slice.T,
                    vmin=np.nanpercentile(df_mode5_stokesVI_slice, 10), vmax=np.nanpercentile(df_mode5_stokesVI_slice, 97),
              # vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode7_stokesVI_slice.index, df_mode7_stokesVI_slice.columns, df_mode7_stokesVI_slice.T,
                    vmin=np.nanpercentile(df_mode7_stokesVI_slice, 10), vmax=np.nanpercentile(df_mode7_stokesVI_slice, 97),
              # vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.set_yscale('log')
fig.colorbar(img, pad=0.02)

# Define the custom ticks
custom_ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]
ax.set_yticks(custom_ticks)
ax.set_yticklabels([str(tick) for tick in custom_ticks])
ax.xaxis.set_minor_locator(AutoMinorLocator(n=5))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_ylim(ax.get_ylim()[::-1])
fig.savefig(f'{path}/plots/ilofar_{str(df_mode3_stokesVI_slice.index[0].date())}_{feature_type}_stokesVI_backgroundSubtracted.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
# Load the dataframes from the pickle files
Tres = 10

df_mode3_stokesI = pd.read_pickle(f'{path}/df_mode3_{Tres}ms_stokesI.pkl')
df_mode5_stokesI = pd.read_pickle(f'{path}/df_mode5_{Tres}ms_stokesI.pkl')
df_mode7_stokesI = pd.read_pickle(f'{path}/df_mode7_{Tres}ms_stokesI.pkl')
time_stokesI = np.load(f'{path}/time_{Tres}ms_stokesI.npy', allow_pickle=True)

# Ensure the 'time' column is in datetime format
df_mode3_stokesI.index = pd.to_datetime(time_stokesI)
df_mode5_stokesI.index = pd.to_datetime(time_stokesI)
df_mode7_stokesI.index = pd.to_datetime(time_stokesI)

df_mode3_stokesV = pd.read_pickle(f'{path}/df_mode3_{Tres}ms_stokesV.pkl')
df_mode5_stokesV = pd.read_pickle(f'{path}/df_mode5_{Tres}ms_stokesV.pkl')
df_mode7_stokesV = pd.read_pickle(f'{path}/df_mode7_{Tres}ms_stokesV.pkl')
time_stokesV = np.load(f'{path}/time_{Tres}ms_stokesV.npy', allow_pickle=True)

# Ensure the 'time' column is in datetime format
df_mode3_stokesV.index = pd.to_datetime(time_stokesV)
df_mode5_stokesV.index = pd.to_datetime(time_stokesV)
df_mode7_stokesV.index = pd.to_datetime(time_stokesV)

df_mode3_stokesVI = df_mode3_stokesV / df_mode3_stokesI
df_mode5_stokesVI = df_mode5_stokesV / df_mode5_stokesI
df_mode7_stokesVI = df_mode7_stokesV / df_mode7_stokesI

# downsample the data
df_mode3_stokesVI = df_mode3_stokesVI.resample('100ms').mean()
df_mode5_stokesVI = df_mode5_stokesVI.resample('100ms').mean()
df_mode7_stokesVI = df_mode7_stokesVI.resample('100ms').mean()


# Slice the DataFrame between start_date and end_date
start_date = None #'2024-05-14 17:30:00'

df_mode3_stokesVI_slice = df_mode3_stokesVI.loc[start_date:]
df_mode5_stokesVI_slice = df_mode5_stokesVI.loc[start_date:]
df_mode7_stokesVI_slice = df_mode7_stokesVI.loc[start_date:]



fig = plt.figure(figsize=[12,6])
ax = fig.add_subplot(111)
img = ax.pcolormesh(df_mode3_stokesVI_slice.index, df_mode3_stokesVI_slice.columns, df_mode3_stokesVI_slice.T,
                    vmin=np.nanpercentile(df_mode3_stokesVI_slice, 10), vmax=np.nanpercentile(df_mode3_stokesVI_slice, 97),
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode5_stokesVI_slice.index, df_mode5_stokesVI_slice.columns, df_mode5_stokesVI_slice.T,
                    vmin=np.nanpercentile(df_mode5_stokesVI_slice, 10), vmax=np.nanpercentile(df_mode5_stokesVI_slice, 97),
                    cmap='RdYlBu_r')
ax.pcolormesh(df_mode7_stokesVI_slice.index, df_mode7_stokesVI_slice.columns, df_mode7_stokesVI_slice.T,
                    vmin=np.nanpercentile(df_mode7_stokesVI_slice, 10), vmax=np.nanpercentile(df_mode7_stokesVI_slice, 97),
                    cmap='RdYlBu_r')
ax.set_yscale('log')
fig.colorbar(img, pad=0.02)

# Define the custom ticks
custom_ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]
ax.set_yticks(custom_ticks)
ax.set_yticklabels([str(tick) for tick in custom_ticks])
ax.xaxis.set_minor_locator(AutoMinorLocator(n=5))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_ylim(ax.get_ylim()[::-1])
fig.savefig(f'{path}/plots/ilofar_{str(df_mode3_stokesVI_slice.index[0].date())}_{feature_type}_stokesVI.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
df_mode3_stokesVI_slice.shape, df_mode5_stokesVI_slice.shape, df_mode7_stokesVI_slice.shape

In [ ]:
np.zeros((df_mode3_stokesVI_slice.shape[0], df_mode3_stokesVI_slice.shape[1] + df_mode5_stokesVI_slice.shape[1] + df_mode7_stokesVI_slice.shape[1])).shape

In [ ]:
combined_df = pd.concat([df_mode3_stokesVI_slice, df_mode3_stokesVI_slice, df_mode7_stokesVI_slice], axis=1)
combined_df.shape

In [ ]:
combined_df.head()

In [ ]:
plt.plot(combined_df.columns)

In [ ]:
combined_df = []
combined_df.append(df_mode7_stokesVI_slice)
combined_df.append(df_mode3_stokesVI_slice)
combined_df.append(df_mode3_stokesVI_slice)

In [ ]:
merged_df = pd.concat(combined_df, axis=1)

# Add NaN for missing values
merged_df = merged_df.apply(lambda x: x if isinstance(x, pd.Series) else np.nan)

merged_df.shape

In [ ]:
min_range = 0
max_range = 5

fig = plt.figure(figsize=[12,6])
ax = fig.add_subplot(111)
ax.pcolormesh(merged_df.index, merged_df.columns, merged_df.T,
                    vmin=min_range, vmax=max_range,
                    cmap='RdYlBu_r')
ax.set_yscale('log')

# Define the custom ticks
custom_ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]
ax.set_yticks(custom_ticks)
ax.set_yticklabels([str(tick) for tick in custom_ticks])
ax.xaxis.set_minor_locator(AutoMinorLocator(n=5))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_ylim(ax.get_ylim()[::-1])
plt.show()